# Intialize notebook

In [2]:
import os
import time
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from functools import reduce
from sklearn.metrics import mean_squared_error

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
%matplotlib inline

from tqdm import tqdm
import pickle

from PIL import Image

import sys
sys.path.insert(0, './python_files/yolo_load_files')
from predictor import get_estimations
sys.path.insert(1, '../src')
from bb2d_detector_yolo import Yolo_Detector

In [3]:
import tkinter

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
# Constants
#KITTI_DATASET = '/media/javier/HDD_linux/KITTI_dataset/training/'
KITTI_DATASET = '/media/robesafe/SSD_SATA/KITTI_DATASET/'
KITTI_DATASET_IMAGES = KITTI_DATASET+'image_2/'
WORK_PATH = os.getcwd()+"/.."
#WEIGHTS_PATH = os.getcwd()+'/../weights'
WEIGHTS_PATH = os.getcwd()+"/../ultralytics_yolov5_master/runs/train/yolov5m_ft/weights/best.pt"
CONF_PATH = os.getcwd()+"/../ultralytics_yolov5_master/data/custom.yaml"

# Run YOLOv5

In [7]:
# Load the model
yolo_detector = Yolo_Detector(WEIGHTS_PATH, CONF_PATH)
yolo_detector.build_model()

YOLOv5 🚀 2022-5-6 torch 1.10.2 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11175MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 20861016 parameters, 0 gradients, 48.0 GFLOPs


In [38]:
# Obtain the detections
n_images = len(os.listdir(KITTI_DATASET_IMAGES))
#n_images = 100
detections = None
for _, i, image_path in zip(tqdm(range(n_images)), range(n_images), sorted(os.listdir(KITTI_DATASET_IMAGES))[:n_images]):
    img = np.array(Image.open(KITTI_DATASET_IMAGES+image_path))
    pred = yolo_detector.detect_yolo(img, 0.6)
    pred = np.array(pred[0].cpu())
    n_pred = pred.shape[0]
    frame = np.array([[i]*n_pred]).T
    ids = np.array([range(n_pred)]).T
    pred = np.append(pred, frame, axis=1)
    pred = np.append(pred, ids, axis=1)
    if detections is not None:
        detections = np.append(detections, pred, axis=0)
    else:
        detections = pred

100%|██████████| 7481/7481 [02:08<00:00, 58.04it/s]


In [39]:
detections

array([[        712,         143,         807, ...,           1,           0,           0],
       [        392,         183,         431, ...,           0,           1,           0],
       [        685,         164,         698, ...,           2,           1,           1],
       ...,
       [        344,         182,         492, ...,           0,        7480,           9],
       [          0,         195,          79, ...,           0,        7480,          10],
       [        624,         179,         655, ...,           0,        7480,          11]])

In [48]:
# Create the dataframe
names = ['Car', 'Pedestrian', 'Cyclist']
df_detections = pd.DataFrame(data=detections,
                             columns=['left','top','right','bottom','score','type','frame','id'])
df_detections['type'] = list(map(lambda x: names[int(x)], df_detections['type'].tolist()))

In [49]:
df_detections

,left,top,right,bottom,score,type,frame,id
0,712.0,143.0,807.0,304.0,0.924625,Pedestrian,0.0,0.0
1,392.0,183.0,431.0,204.0,0.889262,Car,1.0,0.0
2,685.0,164.0,698.0,197.0,0.821587,Cyclist,1.0,1.0
3,669.0,191.0,709.0,225.0,0.921215,Car,2.0,0.0
4,621.0,183.0,734.0,289.0,0.957871,Car,3.0,0.0
...,...,...,...,...,...,...,...,...
32762,510.0,182.0,532.0,230.0,0.859630,Cyclist,7480.0,7.0
32763,758.0,180.0,959.0,256.0,0.846454,Car,7480.0,8.0
32764,344.0,182.0,492.0,230.0,0.835807,Car,7480.0,9.0
32765,0.0,195.0,79.0,370.0,0.820598,Car,7480.0,10.0


In [5]:
#yolo_estimations = get_estimations(KITTI_DATASET, WORK_PATH, WEIGHTS_PATH, batch_size=400, threshold=0.5)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /home/robesafe/Javier/3D-detection-system-lidar-camera/Distance approximation/../master.zip
YOLOv5 🚀 2022-5-3 torch 1.10.2 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11175MiB)



Python 3.7.0 required by YOLOv5, but Python 3.6.13 is currently installed


Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.1 GFLOPs
Adding AutoShape... 


Using previous model KITTI_100_YOLO5l.pkl
Batch: 0 (0.0%)
Batch: 1 (5.346878759524127%)
Batch: 2 (10.693757519048255%)
Batch: 3 (16.040636278572382%)
Batch: 4 (21.38751503809651%)
Batch: 5 (26.73439379762064%)
Batch: 6 (32.081272557144764%)
Batch: 7 (37.428151316668895%)
Batch: 8 (42.77503007619302%)
Batch: 9 (48.12190883571715%)
Batch: 10 (53.46878759524128%)
Batch: 11 (58.815666354765405%)
Batch: 12 (64.16254511428953%)
Batch: 13 (69.50942387381366%)
Batch: 14 (74.85630263333779%)
Batch: 15 (80.20318139286192%)
Batch: 16 (85.55006015238604%)
Batch: 17 (90.89693891191017%)
Last batch


In [6]:
#yolo_estimations

[[[(712, 150), (805, 305), 'person', 0.9119597673416138],
  [(906, 194), (960, 278), 'bicycle', 0.6458708643913269],
  [(252, 211), (312, 283), 'bicycle', 0.6309128403663635],
  [(943, 195), (1000, 277), 'bicycle', 0.6024137735366821],
  [(355, 205), (395, 279), 'bicycle', 0.5679535269737244],
  [(376, 206), (420, 268), 'bicycle', 0.5485382676124573],
  [(316, 204), (368, 280), 'bicycle', 0.53650963306427],
  [(852, 203), (914, 274), 'bicycle', 0.5344498753547668]],
 [[(387, 180), (422, 198), 'car', 0.7156111001968384]],
 [[(658, 189), (703, 222), 'car', 0.5059857964515686]],
 [[(614, 184), (722, 281), 'car', 0.8619185090065002],
  [(0, 234), (209, 373), 'car', 0.6632351279258728]],
 [[(280, 184), (342, 212), 'car', 0.8796266317367554],
  [(365, 181), (404, 203), 'car', 0.7628070116043091],
  [(1211, 262), (1241, 324), 'bicycle', 0.7346388697624207],
  [(403, 171), (437, 192), 'car', 0.6352708339691162]],
 [[(329, 179), (355, 236), 'person', 0.7628330588340759]],
 [[(508, 171), (574, 2

In [7]:
"""
yolo_detecctions = []
n_image = 0
for image in yolo_estimations:
    n_detecction = 0
    for detecction in image:
        obj_type = None
        if detecction[2] == "car":
            obj_type = "Car"
        elif detecction[2] == "person":
            obj_type = "Pedestrian"
        elif detecction[2] == "bicycle":
            obj_type = "Cyclist"
        else:
            continue
        yolo_detecctions.append([n_image, n_detecction, obj_type, detecction[0][0], detecction[0][1],
                                 detecction[1][0], detecction[1][1], detecction[3]])
        n_detecction += 1
    n_image += 1
"""

In [8]:
#df_detections = pd.DataFrame(columns=['frame','id','type','left','top','right','bottom','score'], data=yolo_detecctions)
#df_detections

,frame,id,type,left,top,right,bottom,score
0,0,0,Pedestrian,712,150,805,305,0.911960
1,0,1,Cyclist,906,194,960,278,0.645871
2,0,2,Cyclist,252,211,312,283,0.630913
3,0,3,Cyclist,943,195,1000,277,0.602414
4,0,4,Cyclist,355,205,395,279,0.567954
...,...,...,...,...,...,...,...,...
37256,7480,8,Car,752,183,873,248,0.673685
37257,7480,9,Car,611,175,641,198,0.650878
37258,7480,10,Car,352,178,465,218,0.638382
37259,7480,11,Car,434,178,493,209,0.618590


In [50]:
def get_bounding_boxes_2D(df, image_id):
    df_frame = df[df['frame'] == image_id]
    bbs = []
    for index, row in df_frame.iterrows():
        bb = (row['left'], row['top'], row['right'], row['bottom'], row['type'])
        bbs.append(bb)
    return bbs

def show_bounding_boxes_2D(df, image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    img = KITTI_DATASET+'images/'+name+'.png'
    
    # do projection staff
    plt.figure(figsize=(12,5),dpi=96,tight_layout=True)
    png = mpimg.imread(img)
    IMG_H,IMG_W,_ = png.shape
    # restrict canvas in range
    plt.axis([0,IMG_W,IMG_H,0])
    plt.imshow(png)
    
    # draw bounding boxes
    bbs = get_bounding_boxes_2D(df, image_id)
    for bb in bbs:
        if bb[4] == 'Car':
            color = 'red'
        elif bb[4] == 'Pedestrian':
            color = 'green'
        elif bb[4] == 'Cyclist':
            color = 'blue'
        else:
            color = 'yellow'
        plt.gca().add_patch(Rectangle((bb[0], bb[1]),bb[2]-bb[0],bb[3]-bb[1],linewidth=1,edgecolor=color,facecolor='none'))
    
    # plt.savefig(WORK_PATH+'/yolo_images/{}.png'.format(name),bbox_inches='tight')
    plt.show()

In [51]:
show_bounding_boxes_2D(df_detections, 30)

/home/robesafe/anaconda3/envs/javi_tfm/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


# Associate GT with YOLOv5 detections

In [52]:
df_gt = pd.read_csv(KITTI_DATASET+'kitti_gt.csv')

In [53]:
df_gt

,frame,id,type,truncated,occluded,alpha,left,top,right,bottom,height,width,length,x,y,z,ry,distance,height_image,width_image,theta_ray,incomplete_2d_horizontal,incomplete_2d_bottom
0,0,0,Pedestrian,0.0,0.0,-0.20,712.40,143.00,810.73,307.92,1.89,0.48,1.20,1.84,1.47,8.41,0.01,8.733533,164.92,98.33,0.21,False,False
1,1,0,Car,0.0,0.0,1.85,387.63,181.54,423.81,203.12,1.67,1.87,3.69,-16.53,2.39,58.49,1.57,60.827897,21.58,36.18,-0.28,False,False
2,1,1,Cyclist,0.0,3.0,-1.65,676.60,163.95,688.98,193.93,1.86,0.60,2.02,4.59,1.32,45.84,-1.55,46.088134,29.98,12.38,0.10,False,False
3,2,0,Car,0.0,0.0,-1.67,657.39,190.13,700.07,223.39,1.41,1.58,4.36,3.18,2.27,34.38,-1.58,34.601296,33.26,42.68,0.09,False,False
4,3,0,Car,0.0,0.0,1.55,614.24,181.78,727.31,284.77,1.57,1.73,4.15,1.00,1.75,13.22,1.62,13.372767,102.99,113.07,0.07,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34851,7480,9,Car,0.0,2.0,-0.33,749.78,170.88,917.51,243.48,1.77,1.69,3.82,5.72,1.75,18.85,-0.04,19.776334,72.60,167.73,0.29,False,False
34852,7480,10,Car,0.0,1.0,-2.90,344.07,180.05,479.10,224.15,1.49,1.59,4.44,-7.12,1.78,26.03,3.12,27.044846,44.10,135.03,-0.26,False,False
34853,7480,11,Car,0.0,2.0,-2.91,382.70,179.71,495.48,219.10,1.48,1.63,4.10,-6.79,1.78,28.78,-3.14,29.623654,39.39,112.78,-0.23,False,False
34854,7480,12,Car,0.0,0.0,-1.69,631.71,179.85,678.40,214.37,1.41,1.56,4.08,1.90,1.74,31.98,-1.63,32.083610,34.52,46.69,0.06,False,False


In [54]:
def bbox_2d_overlap(bbox_gt, bbox_det):
    """
    Calculate the overlap between two bounding boxes in 2D.

    :param bbox_gt: (xmin, ymin, xmax, ymax)
    :param bbox_det: (xmin, ymin, xmax, ymax)
    :return: overlap: float
    """
    # Calculate the area of the bounding boxes
    area_gt = (bbox_gt[2] - bbox_gt[0]) * (bbox_gt[3] - bbox_gt[1])
    area_det = (bbox_det[2] - bbox_det[0]) * (bbox_det[3] - bbox_det[1])
    # Calculate the intersection of the areas
    intersection = bbox_2d_intersection(bbox_gt, bbox_det)
    # Calculate the union of the areas
    union = area_gt + area_det - intersection
    # Calculate the overlap
    overlap = intersection / union
    #print(overlap)
    #print("#################")
    return overlap

def bbox_2d_intersection(bbox_gt, bbox_det):
    """
    Calculate the intersection of two bounding boxes in 2D.

    :param bbox_gt: (xmin, ymin, xmax, ymax)
    :param bbox_det: (xmin, ymin, xmax, ymax)
    :return: intersection: float
    """
    # Calculate the intersection
    intersection = max(0, min(bbox_gt[2], bbox_det[2]) - max(bbox_gt[0], bbox_det[0])) * \
                   max(0, min(bbox_gt[3], bbox_det[3]) - max(bbox_gt[1], bbox_det[1]))
    #print(bbox_gt)
    #print(bbox_det)
    #print(intersection)
    return intersection

In [55]:
combined_list = []

# Get different values of column 'frame' and sort them
frames = df_gt['frame'].unique()
frames.sort()

# Create loading bar
loading_bar = tqdm(total=len(frames))

# Iterate over frames
for frame in frames:
    # Iterate over the df_gt rows with the same 'frame' value
    for index_gt, row_gt in df_gt[df_gt['frame'] == frame].iterrows():
        best_overlap = None
        # Iterate over the df_detections rows with the same 'frame' value and the same 'type' value
        for index_det, row_det in df_detections[(df_detections['frame'] == frame) &
                                                (df_detections['type'] == row_gt['type'])].iterrows():
            # If row_gt['type'] == 'Car'
            if row_gt['type'] == 'Car':
                # If the bounding boxes 2D have an overlap greater than 0.7
                if bbox_2d_overlap((row_gt['left'], row_gt['top'], row_gt['right'], row_gt['bottom']),
                                   (row_det['left'], row_det['top'], row_det['right'], row_det['bottom'])) > 0.7:
                    # Store the best overlaping bounding box
                    best_overlap = row_det
            # If row_gt['type'] == 'Pedestrian' or row_gt['type'] == 'Cyclist'
            else:
                # If the bounding boxes 2D have an overlap greater than 0.5
                if bbox_2d_overlap((row_gt['left'], row_gt['top'], row_gt['right'], row_gt['bottom']),
                                   (row_det['left'], row_det['top'], row_det['right'], row_det['bottom'])) > 0.5:
                    # Store the best overlaping bounding box
                    best_overlap = row_det
        # If best_overlap is None
        if best_overlap is None:
            continue
        # If best_overlap is not None
        else:
            combined_list.append([row_gt['frame'], row_gt['id'], row_gt['type'], row_gt['left'], row_gt['top'],
                                  row_gt['right'], row_gt['bottom'], row_gt['occluded'],
                                  row_gt['truncated'], row_gt['distance'], best_overlap['type'],
                                  best_overlap['left'], best_overlap['top'],
                                  best_overlap['right'], best_overlap['bottom'],
                                  best_overlap['score']])
    # Update loading bar
    loading_bar.update(1)

100%|██████████| 7481/7481 [01:18<00:00, 96.23it/s] 

In [56]:
df_combined = pd.DataFrame(columns=['frame','id','type_gt','left_gt','top_gt','right_gt',
                                    'bottom_gt','occluded_gt','truncated_gt','distance_gt','type_yolo','left_yolo',
                                    'top_yolo','right_yolo','bottom_yolo','score_yolo'],
                           data=combined_list)

In [57]:
df_combined

,frame,id,type_gt,left_gt,top_gt,right_gt,bottom_gt,occluded_gt,truncated_gt,distance_gt,type_yolo,left_yolo,top_yolo,right_yolo,bottom_yolo,score_yolo
0,0,0,Pedestrian,712.40,143.00,810.73,307.92,0.0,0.00,8.733533,Pedestrian,712.0,143.0,807.0,304.0,0.924625
1,3,0,Car,614.24,181.78,727.31,284.77,0.0,0.00,13.372767,Car,621.0,183.0,734.0,289.0,0.957871
2,4,0,Car,280.38,185.10,344.90,215.59,0.0,0.00,41.416148,Car,282.0,186.0,351.0,217.0,0.937666
3,4,1,Car,365.14,184.54,406.11,205.20,0.0,0.00,53.626802,Car,371.0,185.0,411.0,206.0,0.907005
4,5,0,Pedestrian,330.06,178.74,360.77,238.64,0.0,0.00,24.626313,Pedestrian,333.0,177.0,364.0,243.0,0.897488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23108,7480,5,Car,934.82,184.38,1241.00,326.25,3.0,0.19,10.388547,Car,952.0,190.0,1242.0,340.0,0.956393
23109,7480,6,Car,882.79,183.25,1122.39,291.95,2.0,0.00,12.664316,Car,902.0,189.0,1160.0,296.0,0.929274
23110,7480,7,Car,800.16,181.42,1050.12,273.59,2.0,0.00,14.876673,Car,813.0,185.0,1069.0,276.0,0.908621
23111,7480,8,Car,755.13,184.19,967.47,253.54,2.0,0.00,17.490249,Car,758.0,180.0,959.0,256.0,0.846454


# Evaluate final method using YOLOv5 detections

## Create final model pipeline

In [58]:
def objective_0(x):
        return 0

def objective_const(x, a):
        return a

def objective_log_f(x, a, b, c):
    return a * np.log(x)**b + c

def objective_f(x, a, b):
    return a * x + b

def objective_2f(x, a, b, c):
    return a * x**2 + b * x + c

def objective_3f(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

def objective_4f(x, a, b, c, d, e):
    return a * x**4 + b * x**3 + c * x**2 + d * x + e

def objective_5f(x, a, b, c, d, e, f):
    return a * x**5 + b * x**4 + c * x**3 + d * x**2 + e * x + f

def objective_6f(x, a, b, c, d, e, f, g):
    return a * x**6 + b * x**5 + c * x**4 + d * x**3 + e * x**2 + f * x + g

def objective_7f(x, a, b, c, d, e, f, g, h):
    return a * x**7 + b * x**6 + c * x**5 + d * x**4 + e * x**3 + f * x**2 + g * x + h

def objective_8f(x, a, b, c, d, e, f, g, h, i):
    return a * x**8 + b * x**7 + c * x**6 + d * x**5 + e * x**4 + f * x**3 + g * x**2 + h * x + i

def objective_9f(x, a, b, c, d, e, f, g, h, i, j):
    return a * x**9 + b * x**8 + c * x**7 + d * x**6 + e * x**5 + f * x**4 + g * x**3 + h * x**2 + i * x + j

class Regression_model:
    def __init__(self, function, variables):
        self.function = function
        self.variables = variables
    def __str__(self):
        return "function = %s, variables = %s"%(self.function, self.variables)

class Regression_model_class:
    def __init__(self, model_car, model_pedestrian, model_cyclist):
        self.car = model_car
        self.pedestrian = model_pedestrian
        self.cyclist = model_cyclist
    def __str__(self):
        return "car: {%s},\npedestrian: {%s},\ncyclist: {%s}"%(self.car, self.pedestrian, self.cyclist)
        
class Regression_model_bb:
    def __init__(self, bb_complete, bb_incomplete):
        self.bb_complete = bb_complete
        self.bb_incomplete = bb_incomplete
    def __str__(self):
        return "bb_complete: {%s},\nbb_incomplete: {%s}"%(self.bb_complete, self.bb_incomplete)
    
class Regression_model_error:
    def __init__(self, height_distance, height_2d, pointcloud_distance, pointcloud_n_points):
        self.height_distance = height_distance
        self.height_2d = height_2d
        self.pointcloud_distance = pointcloud_distance
        self.pointcloud_n_points = pointcloud_n_points
    def __str__(self):
        return "height_distance: {%s},\nheight_2d: {%s},\npointcloud_distance: {%s},\npointcloud_n_points: {%s}"%\
                                (self.height_distance, self.height_2d, self.pointcloud_distance, self.pointcloud_n_points)

In [59]:
height_model = None
pc_proj_refinement_model = None
error_model = None

with open('height_model.pkl', 'rb') as f:
    height_model = pickle.load(f)
with open('pc_proj_refinement_model.pkl', 'rb') as f:
    pc_proj_refinement_model = pickle.load(f)
with open('error_model.pkl', 'rb') as f:
    error_model = pickle.load(f)
    
distance_models = {'height_model':height_model,
                   'pc_proj_refinement_model':pc_proj_refinement_model,
                   'error_model':error_model}

In [60]:
def approximate_distance_height(type_name, top, bottom, bb_complete, model):
    height = bottom - top
    if type_name == 'Car':
        if bb_complete:
            f = model.car.bb_complete.function
            return f(height, *model.car.bb_complete.variables)
        else:
            f = model.car.bb_incomplete.function
            return f(height, *model.car.bb_incomplete.variables)
    elif type_name == 'Pedestrian':
        f = model.pedestrian.function
        return f(height, *model.pedestrian.variables)
    elif type_name == 'Cyclist':
        f = model.cyclist.function
        return f(height, *model.cyclist.variables)
    else:
        raise ValueError("Type must be Car, Pedestrian or Cyclist")

In [61]:
def get_image(image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    img = KITTI_DATASET+'images/'+name+'.png'
    png = mpimg.imread(img)
    return png

In [62]:
def get_point_cloud_projected(image_id):
    name = '%06d'%image_id # 6 digit zeropadding
    binary = KITTI_DATASET+'velodyne/'+name+'.bin'
    with open(KITTI_DATASET+'calib/'+name+'.txt','r') as f:
        calib = f.readlines()

    # P2 (3 x 4) for left eye (intrinsic matrix in homogeneous coordinates)
    P2 = np.matrix([float(x) for x in calib[2].strip('\n').split(' ')[1:]]).reshape(3,4)
    R0_rect = np.matrix([float(x) for x in calib[4].strip('\n').split(' ')[1:]]).reshape(3,3)
    # Add a 1 in bottom-right, reshape to 4 x 4
    R0_rect = np.insert(R0_rect,3,values=[0,0,0],axis=0)
    R0_rect = np.insert(R0_rect,3,values=[0,0,0,1],axis=1)
    Tr_velo_to_cam = np.matrix([float(x) for x in calib[5].strip('\n').split(' ')[1:]]).reshape(3,4)
    Tr_velo_to_cam = np.insert(Tr_velo_to_cam,3,values=[0,0,0,1],axis=0)

    # read raw data from binary
    scan = np.fromfile(binary, dtype=np.float32).reshape((-1,4))
    points = scan[:, 0:3] # lidar xyz (front, left, up)
    # TODO: use fov filter? 
    velo = np.insert(points,3,1,axis=1).T
    velo = np.delete(velo,np.where(velo[0,:]<0),axis=1)
    
    cam = P2 * R0_rect * Tr_velo_to_cam * velo
    cam = np.delete(cam,np.where(cam[2,:]<0)[1],axis=1)
    # get u,v,z
    cam[:2] /= cam[2,:]
    png = get_image(image_id)
    IMG_H,IMG_W,_ = png.shape
    # filter point out of canvas
    u,v,z = cam
    u_out = np.logical_or(u<0, u>IMG_W)
    v_out = np.logical_or(v<0, v>IMG_H)
    outlier = np.logical_or(u_out, v_out)
    cam = np.delete(cam,np.where(outlier),axis=1)
    return cam

In [63]:
def get_projected_pcs_bb(lefts2d, tops2d, rights2d, bottoms2d, projected_pc):
    projected_pcs_bb = []
    for left, top, right, bottom in zip(lefts2d, tops2d, rights2d, bottoms2d):
        filter_bb = (left < projected_pc[0,:]) & (top < projected_pc[1,:]) &\
            (projected_pc[0,:] < right) & (projected_pc[1,:] < bottom)
        pc_distance_bb = projected_pc[2,:][filter_bb]
        # Points on bb
        projected_pcs_bb.append(pc_distance_bb)
        # Deleting saved points
        filter_intersection = (left < projected_pc[0,:]) &\
                       (top < projected_pc[1,:]) &\
                       (projected_pc[0,:] < right) &\
                       (projected_pc[1,:] < bottom)
        projected_pc = np.concatenate((projected_pc[0,:][~filter_intersection],\
                                       projected_pc[1,:][~filter_intersection],\
                                       projected_pc[2,:][~filter_intersection]), axis=0)
    return projected_pcs_bb

In [64]:
def approximate_distance_pc(bbs_complete, tops2d, bottoms2d, types, projected_pcs_bb, model, height_model):
    # Sort bbs by height (change to sort by regression method using height)
    id_bbs = sorted(list(range(len(tops2d))),
                    key=lambda i: approximate_distance_height(types[i], tops2d[i], bottoms2d[i],
                                                              bbs_complete[i], height_model))
    # Create enumare lists
    types = list(enumerate(types))
    tops2d = list(enumerate(tops2d))
    bottoms2d = list(enumerate(bottoms2d))
    # Order using the enumeration
    types.sort(key=lambda x: id_bbs.index(x[0]))
    tops2d.sort(key=lambda x: id_bbs.index(x[0]))
    bottoms2d.sort(key=lambda x: id_bbs.index(x[0]))
    # Save only the intial lists
    types = list(map(lambda x: x[1], types))
    tops2d = list(map(lambda x: x[1], tops2d))
    bottoms2d = list(map(lambda x: x[1], bottoms2d))
        
    # Function to map all bbs
    def get_distances_pc(type_name, top, bottom, pc_distance_bb):
        if pc_distance_bb.size != 0: # Checks if there's any point on the bb 2D
            # Approximate distance
            distance = np.median(np.squeeze(np.asarray(pc_distance_bb)))
            height = bottom - top
            if type_name == 'Car':
                f = model.car.function
                rectification = f(distance, *model.car.variables)
                distance += rectification
            elif type_name == 'Pedestrian':
                f = model.pedestrian.function
                rectification = f(distance, *model.pedestrian.variables)
                distance += rectification
            elif type_name == 'Cyclist':
                f = model.cyclist.function
                rectification = f(distance, *model.cyclist.variables)
                distance += rectification
            else:
                print("Type must be Car, Pedestrian or Cyclist")
            return distance
        else:
            return -1
    # Map over bbs to get distance
    distances = map(lambda type_name, top2d, bottom2d, pc_distance_bb: get_distances_pc(type_name, top2d, bottom2d, pc_distance_bb),
                    types, tops2d, bottoms2d, projected_pcs_bb)
    
    return distances

In [65]:
def ajust_distance(error_model, height_distance, height_2d, pointcloud_distance, pointcloud_n_points, class_name):
    if pointcloud_distance == None:
        return height_distance
    else:
        height_distance_model, height_2d_model, pointcloud_distance_model, pointcloud_n_points_model = None, None, None, None
        # Obtain regression models based on the object type
        if class_name == 'Car':
            height_distance_model = error_model.height_distance.car
            height_2d_model = error_model.height_2d.car
            pointcloud_distance_model = error_model.pointcloud_distance.car
            pointcloud_n_points_model = error_model.pointcloud_n_points.car
        elif class_name == 'Cyclist':
            height_distance_model = error_model.height_distance.cyclist
            height_2d_model = error_model.height_2d.cyclist
            pointcloud_distance_model = error_model.pointcloud_distance.cyclist
            pointcloud_n_points_model = error_model.pointcloud_n_points.cyclist
        elif class_name == 'Pedestrian':
            height_distance_model = error_model.height_distance.pedestrian
            height_2d_model = error_model.height_2d.pedestrian
            pointcloud_distance_model = error_model.pointcloud_distance.pedestrian
            pointcloud_n_points_model = error_model.pointcloud_n_points.pedestrian
        else:
            raise ValueError("Type must be Car, Pedestrian or Cyclist")   
        # Obtain an approximate error based on different metrics
        height_distance_error = height_distance_model.function(height_distance, *height_distance_model.variables)
        height_2d_error = height_2d_model.function(height_2d, *height_2d_model.variables)
        pointcloud_distance_error = pointcloud_distance_model.function(pointcloud_distance, *pointcloud_distance_model.variables)
        pointcloud_n_points_error = pointcloud_n_points_model.function(pointcloud_n_points, *pointcloud_n_points_model.variables)
        # Obtain final distance to use
        sum_error = height_distance_error + height_2d_error + pointcloud_distance_error + pointcloud_n_points_error
        distance = (1 - (height_distance_error + height_2d_error)/sum_error) * height_distance +\
                   (1 - (pointcloud_distance_error + pointcloud_n_points_error)/sum_error) * pointcloud_distance
        return distance

In [66]:
def approximate_distance(image, pc_projected, types, lefts2d, tops2d, rights2d, bottoms2d, distance_models):
    WIDTH_IMAGE = image.shape[1]
    bbs_complete = list(map(lambda left, right: False if left <= 0 or right >= WIDTH_IMAGE-1 else True,
                       lefts2d, rights2d))
    approx_distances_height = list(map(lambda type_name, top, bottom, bb_complete: approximate_distance_height(type_name, top, bottom, bb_complete, distance_models['height_model']),
                                       types, tops2d, bottoms2d, bbs_complete))
    bbs_pc_n_points = get_projected_pcs_bb(lefts2d, tops2d, rights2d, bottoms2d, pc_projected)
    approx_distances_pc = approximate_distance_pc(bbs_complete, tops2d, bottoms2d, types, bbs_pc_n_points, distance_models['pc_proj_refinement_model'], distance_models['height_model'])
    approx_distances_ensemble = list(map(lambda height_distance, top, bottom, pc_distance, pc_n_points, class_name: ajust_distance(distance_models['error_model'], height_distance, bottom-top, pc_distance, pc_n_points.shape[1], class_name),
                                         approx_distances_height, tops2d, bottoms2d, approx_distances_pc, bbs_pc_n_points, types))
    return approx_distances_ensemble

In [67]:
def approximate_distance_frame(frame, models):
    image = get_image(frame)
    pc_projected = get_point_cloud_projected(frame)
    df_frame = df_combined[df_combined['frame'] == frame]
    types = df_frame['type_yolo'].tolist()
    lefts2d = df_frame['left_yolo'].tolist()
    tops2d = df_frame['top_yolo'].tolist()
    rights2d = df_frame['right_yolo'].tolist()
    bottoms2d = df_frame['bottom_yolo'].tolist()
    distance_models = models
    # Calculate distance approximations
    #start = time.time()
    distance = approximate_distance(image, pc_projected, types, lefts2d, tops2d, rights2d, bottoms2d, distance_models)
    #print(time.time() - start)
    return distance

In [68]:
approximate_distance_frame(7480, distance_models)

[7.811653161432483,
 13.392030525365394,
 7.184443384536754,
 9.47105601942491,
 13.039030603481379,
 15.257688607951636,
 18.0429247507926,
 26.72076502308896]

In [69]:
df_combined

,frame,id,type_gt,left_gt,top_gt,right_gt,bottom_gt,occluded_gt,truncated_gt,distance_gt,type_yolo,left_yolo,top_yolo,right_yolo,bottom_yolo,score_yolo
0,0,0,Pedestrian,712.40,143.00,810.73,307.92,0.0,0.00,8.733533,Pedestrian,712.0,143.0,807.0,304.0,0.924625
1,3,0,Car,614.24,181.78,727.31,284.77,0.0,0.00,13.372767,Car,621.0,183.0,734.0,289.0,0.957871
2,4,0,Car,280.38,185.10,344.90,215.59,0.0,0.00,41.416148,Car,282.0,186.0,351.0,217.0,0.937666
3,4,1,Car,365.14,184.54,406.11,205.20,0.0,0.00,53.626802,Car,371.0,185.0,411.0,206.0,0.907005
4,5,0,Pedestrian,330.06,178.74,360.77,238.64,0.0,0.00,24.626313,Pedestrian,333.0,177.0,364.0,243.0,0.897488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23108,7480,5,Car,934.82,184.38,1241.00,326.25,3.0,0.19,10.388547,Car,952.0,190.0,1242.0,340.0,0.956393
23109,7480,6,Car,882.79,183.25,1122.39,291.95,2.0,0.00,12.664316,Car,902.0,189.0,1160.0,296.0,0.929274
23110,7480,7,Car,800.16,181.42,1050.12,273.59,2.0,0.00,14.876673,Car,813.0,185.0,1069.0,276.0,0.908621
23111,7480,8,Car,755.13,184.19,967.47,253.54,2.0,0.00,17.490249,Car,758.0,180.0,959.0,256.0,0.846454


## Execute final model

In [70]:
distance_approx = list(map(lambda x: approximate_distance_frame(x, distance_models), df_combined['frame'].unique().tolist()))

100%|██████████| 7481/7481 [01:34<00:00, 96.23it/s]

In [71]:
distance_approx

[[9.008064153061321],
 [13.23127250662036],
 [39.182612889395436, 54.315798125668316],
 [21.22468942606044],
 [32.501544051596, 21.77126241319317, 37.35648821705938],
 [25.93946782744933, 35.4731894153293],
 [7.0734908488828,
  6.789944815370998,
  7.608029608986109,
  16.199692318911538,
  31.819752268328813],
 [7.5605395871029275,
  12.623843140653793,
  17.2081738526971,
  24.602024744969153,
  25.847891214808975,
  41.68421634559697],
 [13.089092659678427,
  12.669779980319758,
  24.23820092931471,
  8.967687286738249,
  19.55788244478481],
 [22.104822306776164],
 [7.392045878808981, 8.05759250907157],
 [7.324689222672509],
 [9.804131170971228],
 [16.56921134121893],
 [7.849164846068044,
  14.68826783544771,
  16.920793682053493,
  28.533969976109788,
  25.527876398686622,
  31.03602076101572,
  30.37243592185981],
 [33.21403914832409, 28.94092474467621],
 [52.93204281774577],
 [54.1816547176798],
 [9.36351677200391, 10.96668457943747, 29.01829664507801],
 [9.384584363189214],
 [11

In [72]:
distance_approx = reduce(lambda a, b: a+b, distance_approx)

In [73]:
distance_approx

[9.008064153061321,
 13.23127250662036,
 39.182612889395436,
 54.315798125668316,
 21.22468942606044,
 32.501544051596,
 21.77126241319317,
 37.35648821705938,
 25.93946782744933,
 35.4731894153293,
 7.0734908488828,
 6.789944815370998,
 7.608029608986109,
 16.199692318911538,
 31.819752268328813,
 7.5605395871029275,
 12.623843140653793,
 17.2081738526971,
 24.602024744969153,
 25.847891214808975,
 41.68421634559697,
 13.089092659678427,
 12.669779980319758,
 24.23820092931471,
 8.967687286738249,
 19.55788244478481,
 22.104822306776164,
 7.392045878808981,
 8.05759250907157,
 7.324689222672509,
 9.804131170971228,
 16.56921134121893,
 7.849164846068044,
 14.68826783544771,
 16.920793682053493,
 28.533969976109788,
 25.527876398686622,
 31.03602076101572,
 30.37243592185981,
 33.21403914832409,
 28.94092474467621,
 52.93204281774577,
 54.1816547176798,
 9.36351677200391,
 10.96668457943747,
 29.01829664507801,
 9.384584363189214,
 11.590497004991601,
 8.047969280884885,
 14.1857428999

In [74]:
df_combined['distance_approx'] = distance_approx

In [75]:
df_combined

,frame,id,type_gt,left_gt,top_gt,right_gt,bottom_gt,occluded_gt,truncated_gt,distance_gt,type_yolo,left_yolo,top_yolo,right_yolo,bottom_yolo,score_yolo,distance_approx
0,0,0,Pedestrian,712.40,143.00,810.73,307.92,0.0,0.00,8.733533,Pedestrian,712.0,143.0,807.0,304.0,0.924625,9.008064
1,3,0,Car,614.24,181.78,727.31,284.77,0.0,0.00,13.372767,Car,621.0,183.0,734.0,289.0,0.957871,13.231273
2,4,0,Car,280.38,185.10,344.90,215.59,0.0,0.00,41.416148,Car,282.0,186.0,351.0,217.0,0.937666,39.182613
3,4,1,Car,365.14,184.54,406.11,205.20,0.0,0.00,53.626802,Car,371.0,185.0,411.0,206.0,0.907005,54.315798
4,5,0,Pedestrian,330.06,178.74,360.77,238.64,0.0,0.00,24.626313,Pedestrian,333.0,177.0,364.0,243.0,0.897488,21.224689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23108,7480,5,Car,934.82,184.38,1241.00,326.25,3.0,0.19,10.388547,Car,952.0,190.0,1242.0,340.0,0.956393,9.471056
23109,7480,6,Car,882.79,183.25,1122.39,291.95,2.0,0.00,12.664316,Car,902.0,189.0,1160.0,296.0,0.929274,13.039031
23110,7480,7,Car,800.16,181.42,1050.12,273.59,2.0,0.00,14.876673,Car,813.0,185.0,1069.0,276.0,0.908621,15.257689
23111,7480,8,Car,755.13,184.19,967.47,253.54,2.0,0.00,17.490249,Car,758.0,180.0,959.0,256.0,0.846454,18.042925


In [76]:
df_combined['distance_error'] = df_combined['distance_approx'] - df_combined['distance_gt']

In [77]:
df_combined.plot(x='score_yolo', y='distance_error', kind='scatter', s=10)

<AxesSubplot:xlabel='score_yolo', ylabel='distance_error'>

In [78]:
df_combined

,frame,id,type_gt,left_gt,top_gt,right_gt,bottom_gt,occluded_gt,truncated_gt,distance_gt,type_yolo,left_yolo,top_yolo,right_yolo,bottom_yolo,score_yolo,distance_approx,distance_error
0,0,0,Pedestrian,712.40,143.00,810.73,307.92,0.0,0.00,8.733533,Pedestrian,712.0,143.0,807.0,304.0,0.924625,9.008064,0.274531
1,3,0,Car,614.24,181.78,727.31,284.77,0.0,0.00,13.372767,Car,621.0,183.0,734.0,289.0,0.957871,13.231273,-0.141495
2,4,0,Car,280.38,185.10,344.90,215.59,0.0,0.00,41.416148,Car,282.0,186.0,351.0,217.0,0.937666,39.182613,-2.233535
3,4,1,Car,365.14,184.54,406.11,205.20,0.0,0.00,53.626802,Car,371.0,185.0,411.0,206.0,0.907005,54.315798,0.688996
4,5,0,Pedestrian,330.06,178.74,360.77,238.64,0.0,0.00,24.626313,Pedestrian,333.0,177.0,364.0,243.0,0.897488,21.224689,-3.401624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23108,7480,5,Car,934.82,184.38,1241.00,326.25,3.0,0.19,10.388547,Car,952.0,190.0,1242.0,340.0,0.956393,9.471056,-0.917491
23109,7480,6,Car,882.79,183.25,1122.39,291.95,2.0,0.00,12.664316,Car,902.0,189.0,1160.0,296.0,0.929274,13.039031,0.374715
23110,7480,7,Car,800.16,181.42,1050.12,273.59,2.0,0.00,14.876673,Car,813.0,185.0,1069.0,276.0,0.908621,15.257689,0.381016
23111,7480,8,Car,755.13,184.19,967.47,253.54,2.0,0.00,17.490249,Car,758.0,180.0,959.0,256.0,0.846454,18.042925,0.552676


# MSE of the distance by KITTI difficulties and classes

In [79]:
def define_difficulty(row):
    height = row['bottom_gt'] - row['top_gt']
    occlusion = row['occluded_gt']
    truncation = row['truncated_gt']
    if(height >= 40 and occlusion == 0 and truncation <= 0.15):
        return 0
    elif(height >= 25 and occlusion <= 1 and truncation <= 0.3):
        return 1
    elif(height >= 25 and occlusion <= 2 and truncation <= 0.5):
        return 2
    else:
        return 3

In [80]:
df_combined['difficulty'] = df_combined.apply(lambda row: define_difficulty(row), axis=1)

In [81]:
df_combined

,frame,id,type_gt,left_gt,top_gt,right_gt,bottom_gt,occluded_gt,truncated_gt,distance_gt,type_yolo,left_yolo,top_yolo,right_yolo,bottom_yolo,score_yolo,distance_approx,distance_error,difficulty
0,0,0,Pedestrian,712.40,143.00,810.73,307.92,0.0,0.00,8.733533,Pedestrian,712.0,143.0,807.0,304.0,0.924625,9.008064,0.274531,0
1,3,0,Car,614.24,181.78,727.31,284.77,0.0,0.00,13.372767,Car,621.0,183.0,734.0,289.0,0.957871,13.231273,-0.141495,0
2,4,0,Car,280.38,185.10,344.90,215.59,0.0,0.00,41.416148,Car,282.0,186.0,351.0,217.0,0.937666,39.182613,-2.233535,1
3,4,1,Car,365.14,184.54,406.11,205.20,0.0,0.00,53.626802,Car,371.0,185.0,411.0,206.0,0.907005,54.315798,0.688996,3
4,5,0,Pedestrian,330.06,178.74,360.77,238.64,0.0,0.00,24.626313,Pedestrian,333.0,177.0,364.0,243.0,0.897488,21.224689,-3.401624,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23108,7480,5,Car,934.82,184.38,1241.00,326.25,3.0,0.19,10.388547,Car,952.0,190.0,1242.0,340.0,0.956393,9.471056,-0.917491,3
23109,7480,6,Car,882.79,183.25,1122.39,291.95,2.0,0.00,12.664316,Car,902.0,189.0,1160.0,296.0,0.929274,13.039031,0.374715,2
23110,7480,7,Car,800.16,181.42,1050.12,273.59,2.0,0.00,14.876673,Car,813.0,185.0,1069.0,276.0,0.908621,15.257689,0.381016,2
23111,7480,8,Car,755.13,184.19,967.47,253.54,2.0,0.00,17.490249,Car,758.0,180.0,959.0,256.0,0.846454,18.042925,0.552676,2


In [82]:
difficulties = [0, 1, 2]                       
classes = ["Car","Cyclist","Pedestrian"]
text_difficulties = ["Easy", "Moderate", "Hard"]

print("################################", end="\n\n")

# Iterate over difficulties
for difficulty, text_difficulty in zip(difficulties, text_difficulties):
    # Iterate over classes
    for class_name in classes:                  
        df_difficulty = df_combined[(df_combined['difficulty'] <= difficulty) & (df_combined['type_yolo'] == class_name)]
        start_time = time.time()
        mse = mean_squared_error(df_difficulty['distance_gt'].tolist(),
                                          df_difficulty['distance_approx'].tolist())
        print("--- %s seconds to calculate MSE, %s difficulty, %s class  ---" % (time.time() - start_time, text_difficulty, class_name))

        print("\nMean of all MSE by frame, %s difficulty, %s class = %s\n" % (text_difficulty, class_name, mse))

    print("################################", end="\n\n")

################################

--- 0.0007166862487792969 seconds to calculate MSE, Easy difficulty, Car class  ---

Mean of all MSE by frame, Easy difficulty, Car class = 3.298675492260643

--- 0.0002739429473876953 seconds to calculate MSE, Easy difficulty, Cyclist class  ---

Mean of all MSE by frame, Easy difficulty, Cyclist class = 2.2074203273325868

--- 0.0003936290740966797 seconds to calculate MSE, Easy difficulty, Pedestrian class  ---

Mean of all MSE by frame, Easy difficulty, Pedestrian class = 2.0302717365697283

################################

--- 0.0011858940124511719 seconds to calculate MSE, Moderate difficulty, Car class  ---

Mean of all MSE by frame, Moderate difficulty, Car class = 7.349683168175225

--- 0.00033926963806152344 seconds to calculate MSE, Moderate difficulty, Cyclist class  ---

Mean of all MSE by frame, Moderate difficulty, Cyclist class = 3.787519002481461

--- 0.00045943260192871094 seconds to calculate MSE, Moderate difficulty, Pedestrian cla